In [1]:
# import torch
import numpy as np

In [2]:
from numba import cuda
print(cuda.gpus)

<Managed Device 0>


In [3]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

In [41]:
threadsperblock = 256

N = threadsperblock * 32
C = 5
a = np.random.uniform(-100, 100, (N, C))
x = np.random.uniform(-100, 100, (C))
ids = np.arange(N)
distances = np.zeros(N)

# ac = torch.from_numpy(a)
# xc = torch.from_numpy(x)
# idsc = torch.from_numpy(ids)

In [34]:
import math

# CUDA kernel
@cuda.jit
def distancesKernel(results, data, x, weights, maxDist):
    pos = cuda.grid(1)
    s = 0
    for i in range(data[pos].shape[0]):
        s += ((data[pos][i] - x[i]) * weights) ** 2
    results[pos] = (s ** 0.5) / maxDist

# Host code
blockspergrid = math.ceil(a.shape[0] / threadsperblock)
ac = cuda.to_device(a)
xc = cuda.to_device(x)
distancesKernel[blockspergrid, threadsperblock](distances, ac, xc, 1., 1.)
print(distances)

[200.68561296 240.75441411 130.64599758 ... 187.9341813  219.61043515
 226.65892084]


In [35]:
%%timeit
distancesKernel[blockspergrid, threadsperblock](distances, ac, xc, 1., 1.)

459 µs ± 3.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [39]:
%%timeit
_computeDistances(a, x)

282 µs ± 526 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
ac.cuda()
xc.cuda()
idsc.cuda();

In [28]:
def _computeDistancesCuda(data, x, weights=1., maxDist=1.):
    return (torch.sum(((data - x) * weights) ** 2, axis=1) ** 0.5) / maxDist

def _nnFromDataCuda(ids, data, n=1, ignore=0):
    i = data.topk(n+ignore, largest=False)[ignore:n + ignore]
    return i
    #i = data.argpartition(min(n, data.shape[0]))[:n]
    #i = i[data[i].argsort()]
    #i = i[ignore:min(i.size, n)]
    # return ids[i], data[i]

# distances = _computeDistancesCuda(ac, xc)

In [22]:
def _computeDistances(data, x, weights=1., maxDist=1.):
    return (np.sum(((data - x) * weights) ** 2, axis=1) ** 0.5) / maxDist

def _nnFromData(ids, data, n=1, ignore=0):
    i = data.argpartition(np.arange(ignore, min(n + ignore, data.shape[0])))
    i = i[ignore:n + ignore]
    #i = data.argpartition(min(n, data.shape[0]))[:n]
    #i = i[data[i].argsort()]
    #i = i[ignore:min(i.size, n)]
    return ids[i], data[i]

In [37]:
from numba import jit

@jit(nopython=True)
def distancesNumba(data, x, weights=1., maxDist=1.):
    return (np.sum(((data - x) * weights) ** 2, axis=1) ** 0.5) / maxDist

In [38]:
%%timeit
distancesNumba(a, x)

54.5 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
distances = _computeDistances(a, x)
# result = _nnFromData(ids, distances)

45.8 µs ± 969 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
%%timeit
distances = _computeDistancesCuda(ac, xc)
# result = _nnFromDataCuda(idsc, distances)

98.7 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [36]:
# from knn_cuda import KNN